In [ ]:
!unzip oracle_CV.zip

In [ ]:
# Import the neccessary libraries.
import matplotlib.pyplot as plt
import numpy as np
import cv2
import PIL
import pandas as pd
import os
import json
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

print(tf.__version__)

In [ ]:
# Store the training an testing images and labels with the info in the csv's

# Read the csv's
train_df = pd.read_csv(r"train.csv")
test_df = pd.read_csv(r"test.csv")

# Create the training and testing images and labels
train_images = []
train_labels = []
test_images = []

# Training images and labels
for i in range(len(train_df)):
    img = PIL.Image.open(os.path.join(r"", train_df.iloc[i,1]))
    img = img.convert("RGB") # Convert to RGB. This is needed to preserve the 3 color channels of the image.
    train_images.append(np.array(img)) # dtype=uint8 by default
    train_labels.append(train_df.iloc[i,2])

print(len(train_images))
print(len(train_labels))

# Testing images and labels
for i in range(len(test_df)):
    img = PIL.Image.open(os.path.join(r"", test_df.iloc[i,1]))
    img = img.convert("RGB") # Convert to RGB. This is needed to preserve the 3 channels of the image.
    test_images.append(np.array(img))

print(len(test_images))

In [ ]:
#print(train_images[4520].dtype)
#print(train_images[4520].shape)

# Visualize an image
plt.figure()
plt.imshow(train_images[4529])
plt.colorbar()
plt.grid(False)
plt.show()

In [ ]:
# Define class names for each label
class_names = ['Burger', 'Chicken', 'Donut', 'Fries', 'Sausage',
               'Pizza', 'Sandwich', 'Panini']

In [ ]:
# Resize images (in batches so colab doesn't run out of memory)

batch_size = 5 # Set batch size
target_size = (200, 200) # Desired size for all images

# Calculate number of batches
num_batches = int(np.ceil(len(train_images) / batch_size))

train_images_resized = np.empty((len(train_images), *target_size, 3), dtype=np.uint8) # Empty list to store resized images

for i in range(num_batches):
    start = i * batch_size
    end = min((i + 1) * batch_size, len(train_images))
    train_images_batch = train_images[start:end]
    
    for j, img in enumerate(train_images_batch):
        # Resize each image using cubic interpolation
        res = cv2.resize(img, dsize=target_size, interpolation=cv2.INTER_CUBIC)
        #res = cv2.cvtColor(res, cv2.COLOR_BGR2RGB)
        # Append resized image to resized list
        train_images_resized[start + j] = res

train_images = train_images_resized

In [ ]:
train_labels = np.array(train_labels)

In [ ]:
batch_size = 5 # Set batch size
target_size = (200, 200) # Desired size for all images

# Calculate number of batches
num_batches = int(np.ceil(len(test_images) / batch_size))

test_images_resized = np.empty((len(test_images), *target_size, 3), dtype=np.uint8) # Empty list to store resized images

for i in range(num_batches):
    start = i * batch_size
    end = min((i + 1) * batch_size, len(test_images))
    test_images_batch = test_images[start:end]
    
    for j, img in enumerate(test_images_batch):
        # Resize each image using cubic interpolation
        res = cv2.resize(img, dsize=target_size, interpolation=cv2.INTER_CUBIC)
        #res = cv2.cvtColor(res, cv2.COLOR_BGR2RGB)
        # Append resized image to resized list
        test_images_resized[start + j] = res

test_images = test_images_resized

In [ ]:
#print(train_images[0].shape)
# Visualize an image
plt.figure()
plt.imshow(train_images[566])
plt.colorbar()
plt.grid(False)
plt.show()

In [ ]:
# Visualize 25 images
plt.figure(figsize=(10,10))
x = 190
for i in range(x, x + 25):
    plt.subplot(5,5,i-(x-1))
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[train_labels[i]])
plt.show()


# idx_train 10524 has label 4 when it should be 3

In [ ]:
# Data augmentation layers
# Try different things

data_augmentation = keras.Sequential(
  [
    layers.RandomFlip("horizontal_and_vertical",
                      input_shape=(200,
                                  200,
                                  3)),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomContrast(0.2),
    layers.RandomTranslation(height_factor=0.15, width_factor=0.15)
  ]
)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# Define the early stopping callback
early_stopping = EarlyStopping(
    monitor='val_loss', # Monitor the validation loss
    patience=10, # Number of epochs with no improvement after which training will be stopped
    restore_best_weights=True # Restore the best weights from the best epoch
)

In [ ]:
num_classes = len(class_names)

# Create the model
model = Sequential([
    data_augmentation, # Data augmentation
    layers.Rescaling(1./255, input_shape=(200, 200, 3),),
    layers.Conv2D(16, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(128, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Dropout(0.2),
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dense(num_classes)
])

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_images, 
                    train_labels, 
                    validation_split=0.1, # Use 10% of the training data as validation data
                    epochs=100,
                    callbacks=[early_stopping]) # , validation_data=(test_images, test_labels)

In [ ]:
model.summary()

In [ ]:
history.history

In [ ]:
model.save('model072.h5')

In [ ]:
probability_model = tf.keras.Sequential([model, 
                                         tf.keras.layers.Softmax()])

predictions = probability_model.predict(test_images)

In [ ]:
# Create JSON file with predictions

# Create a dictionary with the predictions
predictions_dict = {}
for i in range(0, len(predictions)):
    predictions_dict[str(i)] = (int)(np.argmax(predictions[i]))

# Save the dictionary to a JSON file
with open('predictions.json', 'w') as f:
    json.dump(predictions_dict, f)

# Ver resultados subiendo el json a la web del reto
#Your F1-score is: 0.764 